In [47]:
import pandas as pd

import pickle
from pathlib import Path

In [40]:
month_dir = Path("/projects/frame_align/data/raw/2023-11-01_2023-11-30/")
img_dir = month_dir / "img_data/"

In [41]:
file_sizes = {}
for img_file in img_dir.glob("*.jpg"):
    file_sizes[img_file.name.split(".")[0]] = img_file.stat().st_size
file_size_df = pd.DataFrame.from_dict(file_sizes, orient='index', columns=['file_size'])

In [42]:
file_size_df = file_size_df[file_size_df['file_size'] <= file_size_df['file_size'].quantile(0.95)]
file_size_df = file_size_df[file_size_df['file_size'] >= 5000]
# file_size_df.sort_values('file_size', ascending=True).hist(bins=5)

In [43]:
img_uuids = list(file_size_df.index)

In [44]:
len(img_uuids)

36601

In [19]:
text_df = pd.read_csv(month_dir/"datawithtopiclabels.csv")

In [22]:
text_df.columns

Index(['topic', 'auto_topic_label', 'id', 'authors', 'date_download',
       'date_modify', 'date_publish', 'description', 'filename', 'image_url',
       'language', 'localpath', 'maintext', 'source_domain', 'title',
       'title_page', 'title_rss', 'url', 'month', 'topic_label'],
      dtype='object')

In [23]:
text_df.dropna(subset=['maintext'], inplace=True)
text_df['text_length'] = text_df['maintext'].apply(lambda x: len(x))

In [26]:
text_df['text_length'].sort_values(ascending=True)

18480       67
16735       67
35120       67
32386       67
4245        67
         ...  
37007    53585
6989     55268
15361    57167
15502    58177
15147    59737
Name: text_length, Length: 38775, dtype: int64

In [31]:
text_df.text_length.quantile(0.05)

155.0

In [32]:
lower_quantile = text_df['text_length'].quantile(0.05)
upper_quantile = text_df['text_length'].quantile(0.95)

text_df = text_df[text_df['text_length'] >= lower_quantile]
text_df = text_df[text_df['text_length'] <= upper_quantile]

In [36]:
text_uuids = list(text_df.id)

In [45]:
common_uuids = set(text_uuids).intersection(set(img_uuids))

In [46]:
len(common_uuids)

32964

In [49]:
pickle.dump(common_uuids, open(month_dir/"selected_uuids.pkl", "wb"))